In [ ]:
import numpy as np
import scipy
import itertools
import  pathlib 

import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
plt.interactive(True)
#%config InlineBackend.figure_format = 'pdf'
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

from matplotlib import gridspec

from IPython.display import set_matplotlib_formats
set_matplotlib_formats('pdf', 'png')
plt.rcParams['savefig.dpi'] = 300

plt.rcParams['figure.autolayout'] = False

plt.rcParams['text.usetex'] = True
plt.rcParams['font.family'] = "sans-serif"
plt.rcParams['font.serif'] = "cm"
plt.rcParams['text.latex.preamble'] = [
    r"\usepackage{subdepth}",
    r"\usepackage{type1cm}",
    r'\usepackage{tgheros}',    # helvetica font
    r'\usepackage{sansmath}',   # math-font matching  helvetica
    r'\sansmath'                # actually tell tex to use it!
#    r'\usepackage{siunitx}',    # micro symbols
#    r'\sisetup{detect-all}',    # force siunitx to use the fonts
]  

print(plt.rcParams['figure.figsize'])


def set_size(column=0):
    if (column==0):
        plt.rcParams['figure.figsize']   = [10,6]
        plt.rcParams['axes.labelsize']   = 18 
        plt.rcParams['axes.titlesize']   = 20 
        plt.rcParams['font.size']        = 16 
        plt.rcParams['lines.linewidth']  = 2.0 
        plt.rcParams['lines.markersize'] = 8 
        plt.rcParams['legend.fontsize']  = 14 
    elif (column==1):
        plt.rcParams['figure.figsize']   = [3.5,3.5*0.6]
        plt.rcParams['axes.labelsize']   = 10 
        plt.rcParams['axes.titlesize']   = 12 
        plt.rcParams['font.size']        = 10 
        plt.rcParams['lines.linewidth']  = 1.5 
        plt.rcParams['lines.markersize'] = 10 
        plt.rcParams['legend.fontsize']  = 10 
    elif (column==2):
        plt.rcParams['figure.figsize']   = [7.0,7.0*0.6]
        plt.rcParams['axes.labelsize']   = 10 
        plt.rcParams['axes.titlesize']   = 12 
        plt.rcParams['font.size']        = 10 
        plt.rcParams['lines.linewidth']  = 2.0 
        plt.rcParams['lines.markersize'] = 10 
        plt.rcParams['legend.fontsize']  = 10 
        
set_size(column=2)

# Load the data 

In [ ]:
dir='../data/d_tor1_tor2/'
N_windows = 1370

w_i = np.array([], dtype="int")
t   = np.array([])
q = {}
q["d"], q["tor1"], q["tor2"] = np.array([]), np.array([]), np.array([])

for window in range(0, N_windows):
    print(window, end=" ")
    try:
        c1, c2, c3, c4 = np.loadtxt(dir + str(window) + ".txt", unpack=True)
    except:
        continue

    w_i  = np.append(w_i, np.full(c1.shape, window))
    t    = np.append(t, c1)
    q["d"]    = np.append(q["d"]   , c2)
    q["tor1"] = np.append(q["tor1"], c3)
    q["tor2"] = np.append(q["tor2"], c4)

    

In [ ]:
#load the rmin
dir = '../data/mindist_r9-33/' 

q["mindist"]=np.full(t.shape, np.nan) 
for window in range(0, N_windows):
    fn = dir + str(window) + ".A.mindist.xvg"
    try:
        c1, c2 = np.loadtxt(fn, unpack=True, comments=["#","@"])
        print(window, end=" ")
    except:
        continue
    # here I match the windows and the times    
    mask = (w_i==window) * np.in1d(t,c1)
    q["mindist"][mask] = c2 

In [ ]:
#load the rmin
dir = '../data/pairdist/' 

q["pairdist"]=np.full([t.shape[0], 1225], np.nan) 
for window in range(0, N_windows):
    try:
        c = np.loadtxt(dir + str(window) + ".dist.xvg", comments=["#","@"])
        c1 = c[:,0] 
        c2 = c[:,1:]
        print(window, end=" ")
    except:
        continue
    # here I match the windows and the times    
    mask = (w_i==window) * np.in1d(t,c1)
    q["pairdist"][mask,:] = c2[np.in1d(c1,t[mask])]    

In [ ]:
weights = np.load("../weights/weights.npy")

In [ ]:
# plot them 
plt.figure()
plt.plot(q["d"])
plt.xlabel(r"sample index")
plt.ylabel(r"r / nm")

plt.figure()
plt.plot(q["tor1"])
plt.xlabel(r"sample index")
plt.ylabel(r"$\mathrm{\Theta}^0$ / $^\circ$")

plt.figure()
plt.plot(q["tor2"])
plt.xlabel(r"sample index")
plt.ylabel(r"$\mathrm{\Theta}^1$ / $^\circ$")

plt.figure()
plt.plot(q["mindist"])
plt.xlabel(r"sample index")
plt.ylabel(r"$\mathrm{r_{min} / nm}$")


In [ ]:
# reduce
mask = np.isfinite(q["mindist"])

q["mindist"] = q["mindist"][mask]
q["d"]       = q["d"]      [mask]  
q["tor1"]    = q["tor1"]   [mask]  
q["tor2"]    = q["tor2"]   [mask] 
w_i          = w_i         [mask] 
t            = t           [mask] 
weights      = weights     [mask]    

print(q["mindist"].shape)

In [ ]:
q["pairdist"] = q["pairdist"][mask,:]

In [ ]:
# reduce
mask = q["mindist"] < 1.0

q["mindist"] = q["mindist"][mask][::1]
q["d"]       = q["d"]      [mask][::1]  
q["tor1"]    = q["tor1"]   [mask][::1]  
q["tor2"]    = q["tor2"]   [mask][::1] 
w_i          = w_i         [mask][::1] 
t            = t           [mask][::1] 
weights      = weights     [mask][::1]    

print(q["mindist"].shape)

q["pairdist"] = q["pairdist"][mask,:][::1]

In [ ]:
# plot them 
plt.figure()
plt.plot(q["d"])
plt.xlabel(r"sample index")
plt.ylabel(r"r / nm")

plt.figure()
plt.plot(q["tor1"])
plt.xlabel(r"sample index")
plt.ylabel(r"$\mathrm{\Theta}^0$ / $^\circ$")

plt.figure()
plt.plot(q["tor2"])
plt.xlabel(r"sample index")
plt.ylabel(r"$\mathrm{\Theta}^1$ / $^\circ$")

plt.figure()
plt.plot(q["mindist"])
plt.xlabel(r"sample index")
plt.ylabel(r"$\mathrm{r_{min} / nm}$")


# Cluster

## Calculate the distance matrix 

In [ ]:
D={}

In [ ]:
plt.hist(q["pairdist"][q["pairdist"]<2].flatten(), bins=100)

In [ ]:
N_pieces = 50 

q_t = q["pairdist"].copy()

#divide into N pieces
_indices = np.arange(q_t.shape[0])
_indices_s = np.array_split(_indices, N_pieces)

_temp = np.full([q_t.shape[0],q_t.shape[0]], np.nan)
 
for ii in range(len(_indices_s)):
    l0 = _indices_s[ii].shape[0]
    print(ii, end=" ")
    q_t_0 = q_t[_indices_s[ii],:].reshape(l0, -1, 1)
    for jj in range(ii, len(_indices_s)):
        #print(ii, jj)
        l1 = _indices_s[jj].shape[0]
        q_t_1 = q_t[_indices_s[jj],:].reshape(l1, -1, 1).T   
        dq = q_t_0 - q_t_1

        pair = tuple(np.meshgrid(_indices_s[ii], _indices_s[jj], indexing='ij'))
        _temp[pair] = np.sum(dq**2.0, axis=1)**0.5    
          

In [ ]:
D["pairdist"] = np.triu(_temp,1) + np.tril(_temp.T,0)

In [ ]:
mat = D["pairdist"][::10, ::10]

plt.figure()
square=mat.shape[1]/mat.shape[0]
plt.imshow(mat, aspect=square, origin='upper',interpolation='none')

plt.gca().xaxis.set_ticks_position('bottom')
plt.ylabel(r"sample index")
plt.xlabel(r"sample index")

cbar=plt.colorbar(fraction=0.046, pad=0.04)
cbar.set_label (r"D")


In [ ]:
D["total"] = D["pairdist"] 

In [ ]:
plt.hist(D["total"].flatten(), bins=100)

## Do the clustering

In [ ]:
np.where(np.isnan(D["total"]))

In [ ]:
from sklearn.cluster import AgglomerativeClustering

model = AgglomerativeClustering(n_clusters=None,
                                     distance_threshold=40, 
                                     affinity="precomputed", 
                                     linkage="complete")

clustering = model.fit(D["total"])

In [ ]:
N_clusters = clustering.n_clusters_
print(N_clusters)

plt.plot(clustering.labels_)

In [ ]:
from scipy.cluster.hierarchy import dendrogram

def plot_dendrogram(model, **kwargs):
    # Create linkage matrix and then plot the dendrogram

    # create the counts of samples under each node
    counts = np.zeros(model.children_.shape[0])
    n_samples = len(model.labels_)
    for i, merge in enumerate(model.children_):
        current_count = 0
        for child_idx in merge:
            if child_idx < n_samples:
                current_count += 1  # leaf node
            else:
                current_count += counts[child_idx - n_samples]
        counts[i] = current_count

    linkage_matrix = np.column_stack([model.children_, model.distances_,
                                      counts]).astype(float)

    # Plot the corresponding dendrogram
    dendrogram(linkage_matrix, **kwargs)

In [ ]:
plt.title('Hierarchical Clustering Dendrogram')
# plot the top three levels of the dendrogram
plot_dendrogram(model, truncate_mode='level', p=3)
plt.xlabel("Number of points in node (or index of point if no parenthesis).")
plt.show()

## Calculate inter- and intra- cluster distances

In [ ]:
norm_cluster = np.full([N_clusters, N_clusters] , np.nan)

for c0 in range(0, N_clusters):
    mask0 = clustering.labels_== c0
    for c1 in range(c0, N_clusters):
        mask1 = clustering.labels_== c1
        mat = D["total"][mask0, :][:, mask1]        
        
        if True:
            plt.figure()
            square=mat.shape[1]/mat.shape[0]
            plt.imshow(mat, aspect=square, origin='upper',interpolation='none')

            plt.gca().xaxis.set_ticks_position('bottom')
            plt.ylabel(r"sample index")
            plt.xlabel(r"sample index")

            cbar=plt.colorbar(fraction=0.046, pad=0.04)
            cbar.set_label (r"D")
        
        
        norm_cluster[c0, c1] = np.sum(mat**2.0)**0.5 / (mat.shape[0]*mat.shape[1])**0.5 
        if c1 > c0:
            norm_cluster[c1, c0] = norm_cluster[c0, c1]
        
      #  print(c0,c1)

In [ ]:
mat = norm_cluster.copy()

plt.figure()
square=mat.shape[1]/mat.shape[0]
plt.imshow(mat, aspect=square, origin='upper',interpolation='none')

plt.gca().xaxis.set_ticks_position('bottom')
plt.ylabel(r"sample index")
plt.xlabel(r"sample index")

cbar=plt.colorbar(fraction=0.046, pad=0.04)
cbar.set_label (r"D")

plt.title(r"Cluster Distance Matrix")

diag = mat.diagonal().reshape(-1,1)
mat /= (diag * diag.T)**0.5

mat = np.log(mat)

plt.figure()
square=mat.shape[1]/mat.shape[0]
plt.imshow(mat, aspect=square, origin='upper',interpolation='none')

plt.gca().xaxis.set_ticks_position('bottom')
plt.ylabel(r"sample index")
plt.xlabel(r"sample index")

cbar=plt.colorbar(fraction=0.046, pad=0.04)
cbar.set_label (r"$\ln(D)$")

plt.title(r"log--Diagonal--Normalized Cluster Distance Matrix")

## Calculate the cluster centroids and centers

### Below I am getting the labels in the order of "closest to the rest"

In [ ]:
center_index = np.full(N_clusters, np.nan).astype(int)

cluster_sorted_index = {}

for c0 in range(0, N_clusters):
    mask0 = clustering.labels_== c0

    mat = D["total"][mask0, :][:, mask0]
    _dist = np.sum(mat**2.0, axis=0)
    
    _index = np.where(mask0)[0]    
    
    cluster_sorted_index[c0] = _index[np.argsort(_dist)]
    
for c0 in range(0, N_clusters):
    center_index[c0] = cluster_sorted_index[c0][0]
    print (c0, 
    w_i         [cluster_sorted_index[c0][0]], 
    t           [cluster_sorted_index[c0][0]],
    q["mindist"][cluster_sorted_index[c0][0]],
    q["d"]      [cluster_sorted_index[c0][0]], 
    q["tor1"]   [cluster_sorted_index[c0][0]]*180/np.pi,  
    q["tor2"]   [cluster_sorted_index[c0][0]]*180/np.pi
    )

In [ ]:
import sys, importlib
from densitytopmf import *
importlib.reload(sys.modules['densitytopmf'])
from densitytopmf import *

In [ ]:
##### calculate the  metadaynamics weights
Temperature = 310
# Boltzmann constant in kJ/(mol.K)
kB = 0.0083144621
beta = 1.0/(kB*Temperature)

In [ ]:
cluster_centers = {}
G = {}

In [ ]:
cluster_centers["mindist"] = np.full([N_clusters, 2], np.nan)
cluster_centers["tor1"]    = np.full([N_clusters, 2], np.nan)
cluster_centers["tor2"]    = np.full([N_clusters, 2], np.nan)
cluster_centers["d"]       = np.full([N_clusters, 2], np.nan)

G["c"] = np.full(N_clusters, np.nan)

for i in range(0, N_clusters):
    mask = clustering.labels_==i
    #plt.plot(mask)
    _temp = do_mean_std(q["mindist"]   [mask], weights[mask,0])
    cluster_centers["mindist"][i,:] = _temp

    _temp = do_mean_std_angle(q["tor1"][mask], weights[mask,0])
    cluster_centers["tor1"]   [i,:] = _temp
    
    _temp = do_mean_std_angle(q["tor2"][mask], weights[mask,0])
    cluster_centers["tor2"]   [i,:] = _temp
    
    _temp = do_mean_std(q["d"]         [mask], weights[mask,0])
    cluster_centers["d"]      [i,:] = _temp
    
    _temp = -np.log(np.sum(weights[mask,0])) / beta
    G["c"][i] = _temp
    
G["c"] -= G["c"].min()

In [ ]:
cluster_centers["mindist", "mean"] = np.full([N_clusters, 1], np.nan)
cluster_centers["tor1"   , "mean"] = np.full([N_clusters, 1], np.nan)
cluster_centers["tor2"   , "mean"] = np.full([N_clusters, 1], np.nan)
cluster_centers["d"      , "mean"] = np.full([N_clusters, 1], np.nan)

cluster_centers["mindist", "std"] = np.full([N_clusters, 1], np.nan)
cluster_centers["tor1"   , "std"] = np.full([N_clusters, 1], np.nan)
cluster_centers["tor2"   , "std"] = np.full([N_clusters, 1], np.nan)
cluster_centers["d"      , "std"] = np.full([N_clusters, 1], np.nan)

G["c", "mean"] = np.full(N_clusters, np.nan)
G["c", "std" ] = np.full(N_clusters, np.nan)



for i in range(clustering.n_clusters_):
    mask = clustering.labels_==i
    #plt.plot(mask)
    _temp0, _temp1 = do_mean_std_bs(q["mindist"]   [mask], weights[mask,1:])
    cluster_centers["mindist", "mean"][i] = _temp0["mean"]
    cluster_centers["mindist", "std" ][i] = _temp0["std" ]

    _temp0         = do_mean_std_angle_bs(q["tor1"][mask], weights[mask,1:])
    cluster_centers["tor1", "mean"][i] = _temp0["mean"]
    cluster_centers["tor1", "std" ][i] = _temp0["std" ]
   
    _temp0         = do_mean_std_angle_bs(q["tor2"][mask], weights[mask,1:])
    cluster_centers["tor2", "mean"][i] = _temp0["mean"]
    cluster_centers["tor2", "std" ][i] = _temp0["std" ]
   
    _temp0, _temp1 = do_mean_std_bs(q["d"]         [mask], weights[mask,1:])
    cluster_centers["d", "mean"][i] = _temp0["mean"]
    cluster_centers["d", "std" ][i] = _temp0["std" ]
    
    _temp = -np.log( np.sum(weights[mask,1:], axis=0) ) / beta
    G["c", "mean"][i] = np.mean(_temp)
    G["c", "std" ][i] = np.std(_temp)
    
G["c", "mean"] -= G["c", "mean"].min()

In [ ]:
print ("{:^6} {:^10} {:^10} {:^10} {:^10} {:^10} {:^10} {:^10} {:^10} {:^10} {:^10} {:^10} {:^10} {:^10} {:^10} {:^10} {:^10} {:^10} {:^10} {:^10}".format(
"cluster",
"DG", "DG bm", "DG bs",
"mindist c", "mindist av", "mindist bm", "mindist bs",
"tor1 c", "tor1 av", "tor1 bm", "tor1 bs",
"tor2 c", "tor2 av", "tor2 bm", "tor2 bs",
"d c", "d av", "d bm", "d bs"
)
)

for i in range(0, N_clusters):
    print (    
    "{:<6} {:>10.3f} {:>10.3f} {:>10.3f} {:>10.3f} {:>10.3f} {:>10.3f} {:>10.3f} {:>10.3f} {:>10.3f} {:>10.3f} {:>10.3f} {:>10.3f} {:>10.3f} {:>10.3f} {:>10.3f} {:>10.3f} {:>10.3f} {:>10.3f} {:>10.3f}".format(
    i, 
    G["c"][i], G["c", "mean"][i], G["c", "std"][i],
    q["mindist"][center_index[i]],            cluster_centers["mindist"][i,0],           cluster_centers["mindist", "mean"][i,0],           cluster_centers["mindist", "std"][i,0],           
    q["tor1"]   [center_index[i]]*180/np.pi,  cluster_centers["tor1"]   [i,0]*180/np.pi, cluster_centers["tor1"   , "mean"][i,0]*180/np.pi, cluster_centers["tor1"   , "std"][i,0],    
    q["tor2"]   [center_index[i]]*180/np.pi,  cluster_centers["tor2"]   [i,0]*180/np.pi, cluster_centers["tor2"   , "mean"][i,0]*180/np.pi, cluster_centers["tor2"   , "std"][i,0],    
    q["d"]      [center_index[i]],            cluster_centers["d"]      [i,0],           cluster_centers["d"      , "mean"][i,0],           cluster_centers["d"      , "std"][i,0]
    )
    )

In [ ]:
indices = np.argsort(G["c"])

names = np.around(cluster_centers["d"][:,0], decimals=2).astype(str)

names=np.arange(N_clusters).astype(int)

x = indices.astype(str)
y = G["c"][indices]
yerr = G["c", "std"][indices]

#plt.bar(x, y, alpha=0.1);
plt.errorbar(x, y, yerr=yerr, xerr=None, fmt=' ', capsize = 3)

plt.ylim(0,20)

#plt.xticks(rotation=90)

#plt.xticks(rotation=90)
#
#x = indices.astype(str)
#y = G["c", "mean"][indices]
#yerr = G["c", "std"][indices]
#
#plt.bar(x, y, alpha=0.1)
#plt.errorbar(x, y, yerr=yerr, xerr=None, fmt=' ')

## Make cluster trajectories

In [ ]:
import mdtraj as md

dir = "../data/traj/"
traj = {}

windows = np.unique(w_i)
#print(windows)

for window in windows:
    traj[window] = md.load(dir + str(window) + '.protein.xtc', top=dir + 'topol.pdb')
    print(window, traj[window])

In [ ]:
import nglview as nv 
view = nv.show_mdtraj(traj[1350])  
view

### Generate Cluster Center Trajectory

In [ ]:
traj_center={}
for c0 in range(0, N_clusters):
    w_i_c = w_i [center_index[c0]]
    t_c   = t   [center_index[c0]]
    
    print (c0, w_i_c, t_c)
        
    traj_center[c0] = traj[w_i_c].slice(traj[w_i_c].time == t_c)
    
    print(traj_center[c0])

In [ ]:
import os

outdir = "./cluster-contact-2/"

os.mkdir(outdir)

In [ ]:
from copy import deepcopy

clusters_sorted = np.argsort(G["c"])
print(clusters_sorted)
print(G["c"][clusters_sorted])

for c0 in clusters_sorted:
    oname = "{:.3f}".format(G["c"][c0]) + "_{:0}".format(c0) + ".center.pdb"
    print(oname)
    bfactors =  np.ones(traj_center[c0].n_atoms) # G["c"][c0] 
    traj_center[c0].save_pdb( outdir + oname, bfactors=bfactors)



In [ ]:
print(clusters_sorted)
print(G["c"][clusters_sorted])

lt = [traj_center[key] for key in clusters_sorted]
print(lt)
traj_center_joined = md.join(lt, check_topology=True, discard_overlapping_frames=False)

traj_center_joined.superpose(traj_center_joined)
print(traj_center_joined)

In [ ]:
import nglview as nv 
view = nv.show_mdtraj(traj_center_joined)  
view

### Generate Cluster Trajectories

The structures are resorted based on their distance to the rest. It starts with the closest to the rest, which is also the centroid and moves on.

In [ ]:
traj_allcl={}

for c0 in range(0, N_clusters):
    w_i_c = w_i [cluster_sorted_index[c0]]
    t_c   = t   [cluster_sorted_index[c0]]
    
    ii = 0
    
    traj_allcl[c0] = {}
    traj_allcl[c0][ii] = deepcopy(traj[w_i_c[ii]].slice(traj[w_i_c[ii]].time == t_c[ii]))
    
    for ii in range(1,len(w_i_c)):
        #print(w_i_c[ii], t_c[ii])
        traj_allcl[c0][ii] = deepcopy(traj[w_i_c[ii]].slice(traj[w_i_c[ii]].time == t_c[ii]))
    
    print(traj_allcl[c0])

In [ ]:
clusters_sorted = np.argsort(G["c"])
print(clusters_sorted)
print(G["c"][clusters_sorted])

traj_allcl_joined={}

for c0 in clusters_sorted:
    lt = [traj_allcl[c0][key] for key in traj_allcl[c0].keys()]
    traj_allcl_joined[c0] = md.join(lt, check_topology=True, discard_overlapping_frames=False)
    traj_allcl_joined[c0].superpose(traj_center_joined, frame=c0)

    print(traj_allcl_joined[c0])

In [ ]:
c0 = clusters_sorted[1]
print(c0)
import nglview as nv 
view = nv.show_mdtraj(traj_allcl_joined[c0])  
view

In [ ]:
for c0 in clusters_sorted:
    oname = "{:.3f}".format(G["c"][c0]) + "_{:0}".format(c0) + ".cluster.xtc"
    print(oname)
    traj_allcl_joined[c0].save_xtc(outdir + oname)



# Pair Distances($\mathrm{residue_i}$, $\mathrm{residue_j}$, cluster)

In [ ]:
# plot them 
plt.figure()

q_t = q["pairdist"].copy()
q_t [q_t==0] = np.nan

plt.matshow(q_t, aspect=q_t.shape[1]/q_t.shape[0])
plt.xlabel(r"pairdist")
plt.ylabel(r"sample index")

In [ ]:
q1_t = q["pairdist"]
q2_t = clustering.labels_

mask = np.isfinite(q1_t[:,0])

i=0

A , bin_centers = do_average(y=q1_t[mask,i], x=q2_t[mask], weights=weights[mask,0], bins=np.arange(0,N_clusters+1,1))

for i in range(1,1225):
    y, x = do_average(y=q1_t[mask,i], x=q2_t[mask], weights=weights[mask,0], bins=np.arange(0,N_clusters+1,1))
    A = np.vstack([A, y])

In [ ]:
print(bin_centers , N_clusters)
plt.figure()

#A[A<0.00001] = np.nan
minx, maxx, miny, maxy = bin_centers.min(), bin_centers.max(), .5, 1295.5 

plt.imshow(A, origin="lower", extent = (minx, maxx, miny, maxy), aspect = (maxx-minx) / (maxy-miny) , interpolation="none" )
cbar=plt.colorbar(fraction=0.046, pad=0.04)
cbar.set_label (r"Pairdist")

plt.xlabel(r"$\mathrm{r}$ / nm")
plt.ylabel(r"Residue No.")

In [ ]:
B = A.reshape(35,35,-1)
print(B.shape, B.max(), B.min())

In [ ]:
for i in indices:
    
    plt.figure()

    Z = B[:,:,i].copy()
    
    
    #Z[Z>2] = np.nan

    minx, maxx, miny, maxy = 0.5, 35.5, 0.5, 35.5 
    aspect= (maxx-minx) / (maxy-miny)
    plt.gca().set_aspect(aspect)

    plt.imshow(Z, origin="lower", extent = (minx, maxx, miny, maxy),
               aspect = (maxx-minx) / (maxy-miny),
               interpolation="gaussian", cmap="gist_stern")
    #_X, _Y = np.meshgrid(np.arange(1,36), np.arange(1,36))
    #plt.contourf(_X, _Y, Z.T, 5, alpha=1, cmap="jet_r");

    plt.gca().xaxis.set_ticks_position('bottom')
    
    cbar=plt.colorbar(fraction=0.046, pad=0.04)
    cbar.set_label (r"$\mathrm{d_{ij}}$ / nm")
    
    plt.clim(0.4,2)

    plt.xlabel(r"Residue No. (chain A)")
    plt.ylabel(r"Residue No. (chain B)")
    plt.title(r"$\mathrm{\Delta G} =$ "   + '{0:.2f}'.format(G["c"][i]) + r"$\pm$" + '{0:.2f}'.format(G["c", "std"][i])  + " $\mathrm{kJ/mol}$, " +
               "$\mathrm{cluster} =$ "    + '{0:.0f}'.format(np.arange(0,N_clusters+1,1)[i])         + ", "                   +
               "$\mathrm{r} =$ "          + '{0:.2f}'.format(cluster_centers["d"]      [i,0])           + " $\mathrm{nm}$, "  +
               "$\mathrm{r_{min}} =$ "    + '{0:.2f}'.format(cluster_centers["mindist"][i,0])           + " $\mathrm{nm}$, "  +
               "$\mathrm{{\Theta}_1} =$ " + '{0:.2f}'.format(cluster_centers["tor1"]   [i,0]*180/np.pi) + "$^\circ$, "        +
               "$\mathrm{{\Theta}_2} =$ " + '{0:.2f}'.format(cluster_centers["tor2"]   [i,0]*180/np.pi) + "$^\circ$"
             )

Above plots are fine but they may be unaligned.  Since chain A and B are indistinguishable.  I want to make sure that I average of the aligned contact maps.

In [ ]:
# align
q1_t = q["pairdist"].copy()
q2_t = clustering.labels_.copy()

mask = np.isfinite(q1_t[:,0])
print(q1_t.shape)

y   = q1_t[mask,:].reshape(-1,35,35) 
print(y.shape)

Bn = np.zeros([35,35,N_clusters])

for i in indices:
    print(i)
    mask2 = q2_t==i
    
    w = weights[mask2,0]
    y_m = y[mask2,:,:]
    y_ref = y_m[0,:,:]

    for j in range(0, y_m.shape[0]):
        y_t = y_m[j,:,:]
    
        d0 = np.sum( (y_t   - y_ref)**2 / 2**y_t   )
        d1 = np.sum( (y_t.T - y_ref)**2 / 2**y_t.T )
    
        if d1<d0:
            Bn[:,:,i] += y_t.T*w[j]  
            print(j ,d0, d1, "swapped")
        else:
            Bn[:,:,i] += y_t*w[j]
            print(j ,d0, d1)
    
    Bn[:,:,i]/=np.sum(w)
    


In [ ]:
Bn_0 = Bn.copy()

# align to the average
q1_t = q["pairdist"].copy()
q2_t = clustering.labels_.copy()

mask = np.isfinite(q1_t[:,0])
print(q1_t.shape)

y   = q1_t[mask,:].reshape(-1,35,35) 
print(y.shape)

Bn = np.zeros([35,35,N_clusters])

for i in indices:
    print(i)
    mask2 = q2_t==i

    w = weights[mask2,0]
    y_m = y[mask2,:,:]
    y_ref = Bn_0[:,:,i]

    for j in range(0, y_m.shape[0]):
        y_t = y_m[j,:,:]
    
        d0 = np.sum( (y_t   - y_ref)**2 / 2**y_t   )
        d1 = np.sum( (y_t.T - y_ref)**2 / 2**y_t.T )
    
        if d1<d0:
            Bn[:,:,i]+=y_t.T*w[j]    
            print(j ,d0, d1, "swapped")
        else:
            Bn[:,:,i]+=y_t*w[j]
            print(j ,d0, d1)
    
    Bn[:,:,i]/=np.sum(w)

In [ ]:
for i in indices:
    
    plt.figure()

    Z = Bn[:,:,i].copy()
    
    
    #Z[Z>2] = np.nan

    minx, maxx, miny, maxy = 0.5, 35.5, 0.5, 35.5 
    aspect= (maxx-minx) / (maxy-miny)
    plt.gca().set_aspect(aspect)

    plt.imshow(Z, origin="lower", extent = (minx, maxx, miny, maxy),
               aspect = (maxx-minx) / (maxy-miny),
               interpolation="gaussian", cmap="gist_stern")
    #_X, _Y = np.meshgrid(np.arange(1,36), np.arange(1,36))
    #plt.contourf(_X, _Y, Z.T, 5, alpha=1, cmap="jet_r");

    plt.gca().xaxis.set_ticks_position('bottom')
    
    cbar=plt.colorbar(fraction=0.046, pad=0.04)
    cbar.set_label (r"$\mathrm{d_{ij}}$ / nm")
    
    plt.clim(0.4,2)

    plt.xlabel(r"Residue No. (chain A)")
    plt.ylabel(r"Residue No. (chain B)")
    plt.title(r"$\mathrm{\Delta G} =$ "   + '{0:.2f}'.format(G["c"][i]) + r"$\pm$" + '{0:.2f}'.format(G["c", "std"][i])  + " $\mathrm{kJ/mol}$, " +
               "$\mathrm{cluster} =$ "    + '{0:.0f}'.format(np.arange(0,N_clusters+1,1)[i])         + ", "                   +
               "$\mathrm{r} =$ "          + '{0:.2f}'.format(cluster_centers["d"]      [i,0])           + " $\mathrm{nm}$, "  +
               "$\mathrm{r_{min}} =$ "    + '{0:.2f}'.format(cluster_centers["mindist"][i,0])           + " $\mathrm{nm}$, "  +
               "$\mathrm{{\Theta}_1} =$ " + '{0:.2f}'.format(cluster_centers["tor1"]   [i,0]*180/np.pi) + "$^\circ$, "        +
               "$\mathrm{{\Theta}_2} =$ " + '{0:.2f}'.format(cluster_centers["tor2"]   [i,0]*180/np.pi) + "$^\circ$"
             )